# Pengantar _Computer Aided Cell Factory Design_
Oleh: **Matin Nuhamunada, M.Sc.**

_Laboratorium Bioteknologi, Fakultas Biologi, Universitas Gadjah Mada_

_matin_nuhamunada@ugm.ac.id_

## Daftar Isi
* [Pengantar](##-Pengantar)
* [Setting Up Environment](##-Setting-Up-Environment)
* [Database genome-scale metabolic model](##-Mencari-dan-Mengunduh-Genome-Scale-Metabolic-Model)
* Eksplorasi dan Simulasi Model
* [Manipulasi model](##-Manipulasi-Model)
    * Menghapus reaksi
    * Menambah reaksi
    * Mengubah objective model
* Contoh aplikasi
* DD Decaf

## Pengantar
Materi ini ditulis dalam rangka mendampingi workshop Brainstorming BIOS yang diselenggarakan oleh BEM Fakultas Biologi UGM. Workshop ini bertujuan untuk mengenalkan bagaimana CAD (_Computer Aided Design_) diaplikasikan dalam bioteknologi / metabolic engineering. Materi pada workshop ini diringkas dari workshop Metabolic Modeling for Cell Factory Design dari DTU Biosustain, dan selengkapnya dapat diakses di https://biosustain.github.io/cell-factory-design-course/.

[**Slide pengantar workshop dapat diakses di sini.**](cell-factory-design-ugm-2021.pdf)

Karena materi pada workshop ini lebih ke arah praktis, ada baiknya peserta juga mempelajari materi / kuliah [**Transformasi Digital**](https://elok.ugm.ac.id/course/view.php?id=632) (skill dasar Python), [**Bioinformatika**](https://elok.ugm.ac.id/course/view.php?id=1136), dan **Bioteknologi** (Pengantar Systems & Synthetic Biology). Materi kuliah pengantar systems & synthetic biology (archive Kuliah Bioteknologi semester genap 2020) dapat diakses di: http://ugm.id/lectsystembio2020 & http://ugm.id/lectsyntheticbio2020

## Setting Up Environment
* Saya merekomendasikan untuk menggunakan OS Ubuntu/Linux (ex: menggunakan [WSL2](https://docs.microsoft.com/en-us/windows/wsl/install-win10))
* Ikuti petunjuk instalasi Conda di: https://docs.conda.io/projects/conda/en/latest/user-guide/install/index.html#
* Buat environment sesuai petunjuk di https://biosustain.github.io/cell-factory-design-course/setup/
jupyter labextension install @jupyter-widgets/jupyterlab-manager
jupyter labextension install escher

## Database Genome Scale Metabolic Model
* Buka database: http://bigg.ucsd.edu/
* Cari model: **e_coli_core**
* Ada berapa banyak metabolit, reaksi, dan gen pada model tersebut?
* Genom apa yang digunakan sebagai dasar dari model tersebut?
* Reaksi & stoikiometri?
* Escher Map?
* Bandingkan [**e_coli_core**](http://bigg.ucsd.edu/models/e_coli_core) dengan [**iML1515**](http://bigg.ucsd.edu/models/iML1515). Apa perbedaan dari kedua model tersebut? Jumlah Metabolit, Reaksi, Gen? Referensi genom? Escher Map? 

## Eksplorasi dan Simulasi Model

In [1]:
# load library
from cameo import models
from cameo import load_model
from cameo import fba
from escher import Builder

In [2]:
# cari model e coli core
e_coli_core = load_model('e_coli_core')

# ada berapa banyak metabolit & reaksi? Objektif? 
e_coli_core

Name,e_coli_core
Memory address,0x07f2138423220
Number of metabolites,72
Number of reactions,95
Number of groups,0
Objective expression,1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
Compartments,"extracellular space, cytosol"


In [3]:
# ada berapa banyak gen? 
len(e_coli_core.genes)

137

In [4]:
# detail reaksi PFK
e_coli_core.reactions.PFK

Reaction identifier,PFK
Name,Phosphofructokinase
Memory address,0x07f2138243fa0
Stoichiometry,"atp_c + f6p_c --> adp_c + fdp_c + h_c ATP C10H12N5O13P3 + D-Fructose 6-phosphate --> ADP C10H12N5O10P2 + D-Fructose 1,6-bisphosphate + H+"
GPR,b3916 or b1723
Lower bound,0.0
Upper bound,1000.0


In [5]:
# load escher map
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    data=e_coli_core,
)
builder

Builder()

In [6]:
# simulasi model
s = fba(e_coli_core)
s

In [7]:
s.fluxes

PFK         7.477382
PFL         0.000000
PGI         4.860861
PGK       -16.023526
PGL         4.959985
             ...    
NADH16     38.534610
NADTRHD     0.000000
NH4t        4.765319
O2t        21.799493
PDH         9.282533
Name: fluxes, Length: 95, dtype: float64

In [8]:
# map fluxes
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    reaction_data=s.fluxes,
)
builder

Builder(reaction_data={'PFK': 7.477381962160287, 'PFL': 0.0, 'PGI': 4.860861146496827, 'PGK': -16.023526143167…

## Manipulasi Model